# Filter ATL08

1. Get the BBOX by TileID from 3.x notebooks
2. Query CMR for matching ATL08 granules
3. Get the IDs (later download urls)
4. list the csv (or read from the 

In [119]:
import os
import itertools
import copy

#from notebooks.general.covariateutils import get_index_tile
#import notebooks.general.CovariateUtils 
from CovariateUtils import get_index_tile
# import the MAAP package
from maap.maap import MAAP

# create MAAP class
maap = MAAP()


In [15]:
#bbox = '-101.5,45.5,-100.5,46.5'

geojson_path_albers = "/projects/maap-users/alexdevseed/boreal_tiles.gpkg"
layer = "boreal_tiles_albers"
tile_n = 30543

tile_id = get_index_tile(geojson_path_albers, tile_n, buffer=0, layer = layer)

in_bbox = ",".join(str(coord) for coord in tile_id['bbox_4326'])
out_crs = tile_id['tile_crs']
in_bbox

'-116.85483935749697,50.57021397455074,-116.26013039328576,50.94612905730388'

In [140]:
YEARS = [2019, 2020, 2021]
DATE_START = '06-01T00:00:00Z' # SUMMER
DATE_END = '09-30T23:59:59Z' # SUMMER

date_filters = [f'{year}-{DATE_START},{year}-{DATE_END}' for year in YEARS]
date_filters

['2019-06-01T00:00:00Z,2019-09-30T23:59:59Z',
 '2020-06-01T00:00:00Z,2020-09-30T23:59:59Z',
 '2021-06-01T00:00:00Z,2021-09-30T23:59:59Z']

In [141]:
base_query = {
    'short_name':"ATL08",
    'version':"003",
    'bounding_box':in_bbox
}
    
q3 = [build_query(copy.copy(base_query), date_filter) for date_filter in date_filters]
queries = [dict(base_query, temporal=date_filter) for date_filter in date_filters]

In [142]:
# query CMR as many seasons as necessary
result_chain = itertools.chain.from_iterable([maap.searchGranule(**query) for query in queries])

In [143]:
# get the s3 urls for granules we want to process
granules = [item.getDownloadUrl() for item in result_chain]

In [116]:
# now convert each granule to las format, keeping only relevant dimensions
# eventually make this a dps job to be run over hundreds of granules
# granules only ever need to be converted once, unless dimensions change

In [144]:
granules

['s3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190603002339_10040306_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190607001520_10650306_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190621110851_12860302_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190720094445_03410402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190803212724_05620406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190818082057_07830402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190822081238_08440402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190901200329_10040406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190905195508_10650406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/AT

In [197]:
tracks = list(itertools.chain.from_iterable([[f"gt{i}l",f"gt{i}r"] for i in range(1,4)]))

template_dimensions = {
    "X" : "gtx/land_segments/longitude",
    "Y" : "gtx/land_segments/latitude",
    "Z" : "gtx/land_segments/dem_h",
    "HeightAboveGround" : "gtx/land_segments/canopy/h_canopy",
    "ElevationLow" : "gtx/land_segments/terrain/h_te_best_fit",
    "OffsetTime": "gtx/land_segments/delta_time",
    "landcover": "gtx/land_segments/segment_landcover"
}


tmpl_filters = [
    {
      "type": "filters.merge"
    },
    {
        "limits": "HeightAboveGround[:34028235000000000000000000000000000000]",
        "type": "filters.range",
        "tag": "nodata"
    },
    {
        "type": "filters.ferry",
        "dimensions": "=>GpsTime"
    },
    {
        "type": "filters.assign",
        "value": "GpsTime = OffsetTime + 1198800018"
    },
    {
      "type":"filters.colorinterp",
      "ramp":"pestel_shades",
      "dimension":"Z"
    }
]

tmpl_writers = {
        "type" : "writers.las",
        "extra_dims": "all",
        "pdal_metadata": True,
        "minor_version": "4",
        "a_srs": "EPSG:4326"
}

In [199]:
def set_track(item, track):
    return (item[0], item[1].replace('gtx', track))

def build_dimensions(template_dimensions, track):
    dimensions = dict([set_track(item, track) for item in template_dimensions.items()])
    reader = dict(type="readers.hdf",dimensions=dimensions)
    return reader

In [200]:
#set_track(copy.copy(template_reader), tracks[0])

readers = [build_dimensions(template_dimensions, track) for track in tracks]
#for track in tracks:
#    dimensions = dict([set_track(item, track) for item in template_dimensions.items()])
#    reader = dict(type="readers.hdf",dimensions=dimensions)
#    print(reader)
pipeline = readers+tmpl_filters+[tmpl_writers]

In [165]:
import json

with open('hdf-to-las.json') as f:
    data = json.load(f)
print(data)

{'pipeline': [{'type': 'readers.hdf', 'dimensions': {'X': 'gt1r/land_segments/longitude', 'Y': 'gt1r/land_segments/latitude', 'Z': 'gt1r/land_segments/dem_h', 'HeightAboveGround': 'gt1r/land_segments/canopy/h_canopy', 'ElevationLow': 'gt1r/land_segments/terrain/h_te_best_fit', 'OffsetTime': 'gt1r/land_segments/delta_time', 'landcover': 'gt1r/land_segments/segment_landcover'}}, {'type': 'readers.hdf', 'dimensions': {'X': 'gt1l/land_segments/longitude', 'Y': 'gt1l/land_segments/latitude', 'Z': 'gt1l/land_segments/dem_h', 'HeightAboveGround': 'gt1l/land_segments/canopy/h_canopy', 'ElevationLow': 'gt1l/land_segments/terrain/h_te_best_fit', 'OffsetTime': 'gt1l/land_segments/delta_time', 'landcover': 'gt1l/land_segments/segment_landcover'}}, {'type': 'readers.hdf', 'dimensions': {'X': 'gt2r/land_segments/longitude', 'Y': 'gt2r/land_segments/latitude', 'Z': 'gt2r/land_segments/dem_h', 'HeightAboveGround': 'gt2r/land_segments/canopy/h_canopy', 'ElevationLow': 'gt2r/land_segments/terrain/h_te_b

In [203]:
pipeline_json = json.dumps(pipeline)

'[{"type": "readers.hdf", "dimensions": {"X": "gt1l/land_segments/longitude", "Y": "gt1l/land_segments/latitude", "Z": "gt1l/land_segments/dem_h", "HeightAboveGround": "gt1l/land_segments/canopy/h_canopy", "ElevationLow": "gt1l/land_segments/terrain/h_te_best_fit", "OffsetTime": "gt1l/land_segments/delta_time", "landcover": "gt1l/land_segments/segment_landcover"}}, {"type": "readers.hdf", "dimensions": {"X": "gt1r/land_segments/longitude", "Y": "gt1r/land_segments/latitude", "Z": "gt1r/land_segments/dem_h", "HeightAboveGround": "gt1r/land_segments/canopy/h_canopy", "ElevationLow": "gt1r/land_segments/terrain/h_te_best_fit", "OffsetTime": "gt1r/land_segments/delta_time", "landcover": "gt1r/land_segments/segment_landcover"}}, {"type": "readers.hdf", "dimensions": {"X": "gt2l/land_segments/longitude", "Y": "gt2l/land_segments/latitude", "Z": "gt2l/land_segments/dem_h", "HeightAboveGround": "gt2l/land_segments/canopy/h_canopy", "ElevationLow": "gt2l/land_segments/terrain/h_te_best_fit", "O

['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']